#  SUMMOスクレイピングメイン

In [1]:
#必要なライブラリをインポート
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from urllib.parse import urljoin

In [2]:
def Recursive_PageNum(All_PageFullURLs, url):
    #データ取得
    result = requests.get(url)
    c = result.content
    #HTMLを元に、オブジェクトを作る
    soup = BeautifulSoup(c, "html.parser")
    #ページ数を取得
    body = soup.find("body")
    pages = body.find("div",{'class':'pagination pagination_set-nav'}) #Page数の部分のhtmlを抜き出す
    links = pages.select("a[href]") #link付きaタグを抜き出す
    #ページ選択で数値になっているリンクを引っ張ってくる（"次へ"を除く）
    PageURLs = [link.get("href") for link in links if link.get_text().isdigit()] 
    PageFullURLs = [urljoin("https://suumo.jp/", relative) for relative in PageURLs] #相対パス -> 絶対パス
    #1ページ目を先頭に格納
    PageFullURLs.insert(0, url)
    PageFullURLs = list(dict.fromkeys(PageFullURLs)) #重複削除
    nPages = [int(link.get_text()) for link in links if link.get_text().isdigit()]
    All_PageFullURLs.extend(PageFullURLs)
    All_PageFullURLs = list(dict.fromkeys(All_PageFullURLs)) #重複削除
    #再帰関数Exit条件 1 pageしかない場合 / 2 pageしかない場合 / MaxとMax-1番目のページ数の差が1の場合
    if len(nPages) == 0 or len(nPages) == 1 or (sorted(nPages)[-1] - sorted(nPages)[-2]) ==1:
        print("Exit recursive loop! # of URLs are    " + str(len(All_PageFullURLs)))
        time.sleep(5)
        return All_PageFullURLs
    #再帰関数 ページ数が２番目に大きいURLを次の読み込みURLに指定
    time.sleep(5)
    return Recursive_PageNum(All_PageFullURLs, PageFullURLs[-2])

def PageNum_Easy(url):
    #データ取得
    result = requests.get(url)
    c = result.content
    #HTMLを元に、オブジェクトを作る
    soup = BeautifulSoup(c, "html.parser")
    #ページ数を取得
    body = soup.find("body")
    pages = body.find("div",{'class':'pagination pagination_set-nav'}) #Page数の部分のhtmlを抜き出す
    links = pages.select("a[href]") #link付きaタグを抜き出す
    #ページ選択で数値になっているものを引っ張ってくる（"次へ"を除く / 1のみの場合は空リスト）
    nPages = [int(link.get_text()) for link in links if link.get_text().isdigit()]
    if len(nPages) == 0:
        PageFullURLs = [url]
    else:
        MaxPages = sorted(nPages)[-1] #Max page数
        PageFullURLs = []
        for ipg in range(MaxPages):
            base_URL = url + '&page=' + str(ipg+1)
            PageFullURLs.append(base_URL)
    return PageFullURLs

In [3]:
def ParseRoomDetail(EstateElem, url):
    #マンション名取得
    EstateName = EstateElem.find("div",{'class':'cassetteitem_content-title'}).get_text()
    #住所取得
    EstateAddress = EstateElem.find("li",{'class':'cassetteitem_detail-col1'}).get_text()
    #最寄り駅取得
    EstateLocationElem = EstateElem.find("li",{'class':'cassetteitem_detail-col2'}).find_all("div",{'class':'cassetteitem_detail-text'})
    EstateLocations = [EstateLocation.get_text() for EstateLocation in EstateLocationElem] #リストで取得
    EstateLocation = ' --- '.join(EstateLocations)
    #築年数と建物高さを取得
    EstateCol3Elem = EstateElem.find("li",{'class':'cassetteitem_detail-col3'}).find_all("div")
    EstageAge = EstateCol3Elem[0].get_text()
    EstageHight = EstateCol3Elem[1].get_text()
    #Header Info をListに
    HeaderInfo = [EstateName, EstateAddress, EstateLocation, EstageAge, EstageHight, url]
    #階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
    RoomtableElem =  EstateElem.find("table",{'class':'cassetteitem_other'})
    RoomDetail = []
    #Contents 
    for rooms in RoomtableElem.find_all("tbody"):
        Roomtable = [temp.get_text() for temp in rooms.findAll('td')]
        if "cassetteitem_other-checkbox--newarrival" in rooms.td['class']:
            Roomtable.append("New")
        else:
            Roomtable.append("Exsiting")
        Roomlinks = EstateElem.select("a[href]") #link付きaタグを抜き出す
        #"詳細を見る"の表示になっているリンクを引っ張ってくる
        RoomDetailURL = [link.get("href") for link in Roomlinks if link.get_text() == "詳細を見る"][0]
        RoomFullDetailURL = urljoin("https://suumo.jp/", RoomDetailURL) #相対パス -> 絶対パス
        Roomtable.append(RoomFullDetailURL)
        #Add Header Info
        Roomtable.extend(HeaderInfo)
        RoomDetail.append(Roomtable)
    return RoomDetail

In [4]:
def Parsedistrict(PageFullURLs, RoomDetails):
    for icount, url in enumerate(PageFullURLs):
        print("    Room Detail Status: " + str(icount + 1) + "/" + str(len(PageFullURLs)))
        try:
            #データ取得
            result = requests.get(url)
            c = result.content
            #HTMLを元に、オブジェクトを作る
            soup = BeautifulSoup(c, "html.parser")
            #物件リストの部分を切り出し
            summary = soup.find("div",{'id':'js-bukkenList'})
            #マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し - デフォルト設定で最大30件の物件表示
            cassetteitems = summary.find_all("div",{'class':'cassetteitem'})
            #マンションの数でループ
            for EstateElem in cassetteitems:
                RoomDetails.extend(ParseRoomDetail(EstateElem, url))
        except requests.exceptions.RequestException as e:
            print("エラー : ",e)
        time.sleep(5)
    print("total # of Rooms: " + str(len(RoomDetails)))
    return RoomDetails

In [5]:
#検索１ページ目
#ta=が都道府県コード
#sc=を無くせば都道府県単位の情報になる
#東京23区はsc=13101 ~ 13123
#条件：　12万円以下 15分以内 15年以内 2階以上／室内洗濯機置場／バス・トイレ別 管理費・共益費込み 鉄筋系／鉄骨系／ブロック・その他
BaseURLs = [
    #東京都
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13",
    #神奈川県
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14",
    #千葉県
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=12",
    #埼玉県
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=11"
]
    
# # test用 1 page
# BaseURLs = ["https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=12&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&ta=13&cb=0.0&ct=5.0&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="]
# # test用 2 pages
# BaseURLs = ["https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=12&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&ta=13&cb=0.0&ct=7.0&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="]
# #test用 3 pages
# BaseURLs = ["https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=12&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&ta=13&cb=0.0&ct=8.5&co=1&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="]
# #test用 8 pages
# BaseURLs = ["https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=12&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&ta=13&cb=0.0&ct=9.0&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="]
# #test用 35 pages
# BaseURLs = ["https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"]

In [6]:
#Header Name
soup = BeautifulSoup(requests.get(BaseURLs[0]).content, "html.parser")
body = soup.find("body")
RoomtableHeadElem =  body.find("div",{'class':'cassetteitem'}).find("thead").find_all("th")
HeaderNames = [temp.get_text() for temp in RoomtableHeadElem]
HeaderNames.append("NewArrival")
HeaderNames.append("RoomDetailLink")
HeaderNames.extend(["マンション名", "住所", "最寄り駅", "築年数", "建物高さ","SearchURL"])
HeaderNames = [temp.replace("\xa0", str(i)) for i, temp in enumerate(HeaderNames)] #空白の場合行番号で置き換え

In [7]:
#Scraping Main
# RoomDetails = []　　#BaseURLsのデータをまとまりで出したい場合
for iMcount, url in enumerate(BaseURLs):
    print("District Status: " + str(iMcount + 1) + "/" + str(len(BaseURLs)))
    All_PageFullURLs = []
#     All_PageFullURLs = Recursive_PageNum(All_PageFullURLs, url) #全てのページのURLを取得
    All_PageFullURLs = PageNum_Easy(url)
    RoomDetails = [] #データをBaseURLsごと出力
    RoomDetails = Parsedistrict(All_PageFullURLs, RoomDetails) #ページごとの物件情報取得 / Pageのループはこの中
    df = pd.DataFrame(RoomDetails, columns = HeaderNames)
    filename = "SUMMO_FullRoom_" + str(iMcount) + ".csv"
    df.to_csv(filename)

District Status: 1/4
    Room Detail Status: 1/638
    Room Detail Status: 2/638
    Room Detail Status: 3/638
    Room Detail Status: 4/638
    Room Detail Status: 5/638
    Room Detail Status: 6/638
    Room Detail Status: 7/638
    Room Detail Status: 8/638
    Room Detail Status: 9/638
    Room Detail Status: 10/638
    Room Detail Status: 11/638
    Room Detail Status: 12/638
    Room Detail Status: 13/638
    Room Detail Status: 14/638
    Room Detail Status: 15/638
    Room Detail Status: 16/638
    Room Detail Status: 17/638
    Room Detail Status: 18/638
    Room Detail Status: 19/638
    Room Detail Status: 20/638
    Room Detail Status: 21/638
    Room Detail Status: 22/638
    Room Detail Status: 23/638
    Room Detail Status: 24/638
    Room Detail Status: 25/638
    Room Detail Status: 26/638
    Room Detail Status: 27/638
    Room Detail Status: 28/638
    Room Detail Status: 29/638
    Room Detail Status: 30/638
    Room Detail Status: 31/638
    Room Detail Status: 32/

    Room Detail Status: 260/638
    Room Detail Status: 261/638
    Room Detail Status: 262/638
    Room Detail Status: 263/638
    Room Detail Status: 264/638
    Room Detail Status: 265/638
    Room Detail Status: 266/638
    Room Detail Status: 267/638
    Room Detail Status: 268/638
    Room Detail Status: 269/638
    Room Detail Status: 270/638
    Room Detail Status: 271/638
    Room Detail Status: 272/638
    Room Detail Status: 273/638
    Room Detail Status: 274/638
    Room Detail Status: 275/638
    Room Detail Status: 276/638
    Room Detail Status: 277/638
    Room Detail Status: 278/638
    Room Detail Status: 279/638
    Room Detail Status: 280/638
    Room Detail Status: 281/638
    Room Detail Status: 282/638
    Room Detail Status: 283/638
    Room Detail Status: 284/638
    Room Detail Status: 285/638
    Room Detail Status: 286/638
    Room Detail Status: 287/638
    Room Detail Status: 288/638
    Room Detail Status: 289/638
    Room Detail Status: 290/638
    Room

    Room Detail Status: 517/638
    Room Detail Status: 518/638
    Room Detail Status: 519/638
    Room Detail Status: 520/638
    Room Detail Status: 521/638
    Room Detail Status: 522/638
    Room Detail Status: 523/638
    Room Detail Status: 524/638
    Room Detail Status: 525/638
    Room Detail Status: 526/638
    Room Detail Status: 527/638
    Room Detail Status: 528/638
    Room Detail Status: 529/638
    Room Detail Status: 530/638
    Room Detail Status: 531/638
    Room Detail Status: 532/638
    Room Detail Status: 533/638
    Room Detail Status: 534/638
    Room Detail Status: 535/638
    Room Detail Status: 536/638
    Room Detail Status: 537/638
    Room Detail Status: 538/638
    Room Detail Status: 539/638
    Room Detail Status: 540/638
    Room Detail Status: 541/638
    Room Detail Status: 542/638
    Room Detail Status: 543/638
    Room Detail Status: 544/638
    Room Detail Status: 545/638
    Room Detail Status: 546/638
    Room Detail Status: 547/638
    Room

    Room Detail Status: 137/175
    Room Detail Status: 138/175
    Room Detail Status: 139/175
    Room Detail Status: 140/175
    Room Detail Status: 141/175
    Room Detail Status: 142/175
    Room Detail Status: 143/175
    Room Detail Status: 144/175
    Room Detail Status: 145/175
    Room Detail Status: 146/175
    Room Detail Status: 147/175
    Room Detail Status: 148/175
    Room Detail Status: 149/175
    Room Detail Status: 150/175
    Room Detail Status: 151/175
    Room Detail Status: 152/175
    Room Detail Status: 153/175
    Room Detail Status: 154/175
    Room Detail Status: 155/175
    Room Detail Status: 156/175
    Room Detail Status: 157/175
    Room Detail Status: 158/175
    Room Detail Status: 159/175
    Room Detail Status: 160/175
    Room Detail Status: 161/175
    Room Detail Status: 162/175
    Room Detail Status: 163/175
    Room Detail Status: 164/175
    Room Detail Status: 165/175
    Room Detail Status: 166/175
    Room Detail Status: 167/175
    Room

# Debug用 - Room　スクレイピング



url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=12&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&ta=13&cb=0.0&ct=5.0&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="
#データ取得
result = requests.get(url)
c = result.content
#HTMLを元に、オブジェクトを作る
soup = BeautifulSoup(c, "html.parser")
#物件リストの部分を切り出し
summary = soup.find("div",{'id':'js-bukkenList'})
#マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し - デフォルト設定で最大30件の物件表示
cassetteitems = summary.find_all("div",{'class':'cassetteitem'})

EstateElem = cassetteitems[0]

#マンション名取得
EstateName = EstateElem.find("div",{'class':'cassetteitem_content-title'}).get_text()
#住所取得
EstateAddress = EstateElem.find("li",{'class':'cassetteitem_detail-col1'}).get_text()
#最寄り駅取得
EstateLocationElem = EstateElem.find("li",{'class':'cassetteitem_detail-col2'}).find_all("div",{'class':'cassetteitem_detail-text'})
EstateLocations = [EstateLocation.get_text() for EstateLocation in EstateLocationElem] #リストで取得
EstateLocation = ' --- '.join(EstateLocations)
#築年数と建物高さを取得
EstateCol3Elem = EstateElem.find("li",{'class':'cassetteitem_detail-col3'}).find_all("div")
EstageAge = EstateCol3Elem[0].get_text()
EstageHight = EstateCol3Elem[1].get_text()
#Header Info をListに
HeaderInfo = [EstateName, EstateAddress, EstateLocation, EstageAge, EstageHight, url]
#階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
RoomtableElem =  EstateElem.find("table",{'class':'cassetteitem_other'})
RoomDetail = []
#Contents 
for rooms in RoomtableElem.find_all("tbody"):
    Roomtable = [temp.get_text() for temp in rooms.findAll('td')]
    if "cassetteitem_other-checkbox--newarrival" in rooms.td['class']:
        Roomtable.append("New")
    else:
        Roomtable.append("Exsiting")
    Roomlinks = EstateElem.select("a[href]") #link付きaタグを抜き出す
    #"詳細を見る"の表示になっているリンクを引っ張ってくる
    RoomDetailURL = [link.get("href") for link in Roomlinks if link.get_text() == "詳細を見る"][0]
    RoomFullDetailURL = urljoin("https://suumo.jp/", RoomDetailURL) #相対パス -> 絶対パス
    Roomtable.append(RoomFullDetailURL)
    #Add Header Info
    Roomtable.extend(HeaderInfo)
    RoomDetail.append(Roomtable)
RoomDetail

# Debug用

# url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13"
# # test用 1 page
# url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=12&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&ta=13&cb=0.0&ct=5.0&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="
# # test用 2 pages
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=12&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&ta=13&cb=0.0&ct=7.0&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="

#データ取得
result = requests.get(url)
c = result.content
#HTMLを元に、オブジェクトを作る
soup = BeautifulSoup(c, "html.parser")
#ページ数を取得
body = soup.find("body")
pages = body.find("div",{'class':'pagination pagination_set-nav'}) #Page数の部分のhtmlを抜き出す
links = pages.select("a[href]") #link付きaタグを抜き出す

#ページ選択で数値になっているリンクを引っ張ってくる（"次へ"を除く）
PageURLs = [link.get("href") for link in links if link.get_text().isdigit()] 
PageFullURLs = [urljoin("https://suumo.jp/", relative) for relative in PageURLs] #相対パス -> 絶対パス
#1ページ目を先頭に格納
PageFullURLs.insert(0, url)
PageFullURLs = list(dict.fromkeys(PageFullURLs)) #重複削除
nPages = [int(link.get_text()) for link in links if link.get_text().isdigit()] #Max page数
if len(nPages) == 0:
    print(PageFullURLs)
else:
    MaxPages = sorted(nPages)[-1]
    PageFullURLs = []
    for ipg in range(MaxPages):
        base_URL = url + '&page=' + str(ipg+1)
        PageFullURLs.append(base_URL)
PageFullURLs

    
    
#     All_PageFullURLs.extend(PageFullURLs)
#     All_PageFullURLs = list(dict.fromkeys(All_PageFullURLs)) #重複削除
    
    
    
#     #再帰関数Exit条件 1 pageしかない場合 / 2 pageしかない場合 / MaxとMax-1番目のページ数の差が1の場合
#     if len(nPages) == 0 or len(nPages) == 1 or (sorted(nPages)[-1] - sorted(nPages)[-2]) ==1:
#         print("Exit recursive loop! # of URLs are    " + str(len(All_PageFullURLs)))
#         time.sleep(5)
#         return All_PageFullURLs
#     #再帰関数 ページ数が２番目に大きいURLを次の読み込みURLに指定
#     time.sleep(5)
#     return Recursive_PageNum(All_PageFullURLs, PageFullURLs[-2])



# 前処理

filename = "SUMMO_Room.csv"
df = pd.read_csv(filename, index_col=0)
#いらないコラムを削除
df = df.drop(['0', '1','6','8'], axis=1)
df.head()

In [ ]:
# #階数を数字に変換 2-3などとなっている場合は、前半を取得。
# df["階"].extract("+@階", regex=True)


In [ ]:
# df.head()

BaseURLs = [
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #千代田区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #中央区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13103&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #港区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13104&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #新宿区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13105&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #文京区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13113&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #渋谷区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13106&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #台東区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13107&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #墨田区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13108&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #江東区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13118&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #荒川区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13121&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #足立区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13122&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1", #葛飾区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13123&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1" #江戸川区
]

"" #
""

sc=14131
14 --> 神奈川県
1 --> 川崎市
31--> 川崎区

sc=14137
14 --> 神奈川県
1 --> 川崎市
37--> 麻生区

sc=13101

taが都道府県レベル
11が埼玉県
12 が千葉県
13が東京都
14が神奈川県

scを無くせば都道府県単位の情報になる
#12万円以下 15分以内 15年以内 2階以上／室内洗濯機置場／バス・トイレ別 管理費・共益費込み 鉄筋系／鉄骨系／ブロック・その他
東京都
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13
神奈川県
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14
千葉県
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=12
埼玉県
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=11


BaseURLs = [
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14&sc=14131", #川崎市川崎区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14&sc=14132", #川崎市幸区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14&sc=14133", #川崎市中原区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14&sc=14134", #川崎市高津区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14&sc=14135", #川崎市多摩区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14&sc=14136", #川崎市宮前区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=14&sc=14137", #川崎市麻生区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13&sc=13101", #東京都千代田区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13&sc=13300", #西多摩郡
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13&sc=13201", #八王子市
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13&sc=13119", #東京都板橋区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13&sc=13202", #立川市
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=13&sc=13229", #西東京市
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=12&sc=12101", #千葉市中央区
    "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&co=1&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=12.0&et=15&mb=0&mt=9999999&cn=15&ta=11&sc=11101", #さいたま市西区
    "", #
    "", #
    "", #
    "", #
    "", #
]
